# [Create Agent with Function Call](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-projects-readme?view=azure-python-preview#create-agent-with-function-call) using FunctionTool + ToolOutput
You can enhance your Agents by defining **callback functions as function tools**. These can be provided to `create_agent` via either the `toolset` parameter or the **combination** of `tools and tool_resources`. Here are the distinctions:

1. **toolset**: When using the toolset parameter, you provide not only the function definitions and descriptions but also their implementations. The SDK will execute these functions within `create_and_run_process` or `streaming`. These functions will be invoked based on their definitions.
2. **tools and tool_resources**: When using the tools and tool_resources parameters, only the function definitions and descriptions are provided to `create_agent`, without the implementations. The `Run` or event handler of stream will raise a `requires_action` status based on the function definitions. **Your code must handle this status and call the appropriate functions**.<br/>

As a reference point, let's see how we were used to managed this with [OpenAI Assistants API](https://github.com/maurominella/openai/blob/main/assistantapi/Assistant%20APIs%2002%20-%20SDK%20with%20Function%20Calling.ipynb)

# Constants

In [1]:
import os, json, time
from IPython.display import Markdown, display
from dotenv import load_dotenv # requires python-dotenv
from PIL import Image # requires pip install pillow
from datetime import datetime
from common.agents_helper_functions import *
import importlib.metadata
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

if not load_dotenv("./../config/credentials_my.env"):
    print("Environment variables not loaded, cell execution stopped")
else:
    print("Environment variables have been loaded ;-)")

deployment_name =  os.environ["MODEL_DEPLOYMENT_NAME_MINI"]
model_name =  os.environ["MODEL_DEPLOYMENT_NAME"]
api_version = os.environ["OPENAI_API_VERSION"] # at least "2025-03-01-preview"

project_connection_string = os.environ["PROJECT_CONNECTION_STRING"]

print(f'Project Connection String: <...{project_connection_string[-30:]}>')
version = importlib.metadata.version("azure-ai-projects")
print(f"azure-ai-projects library installed version: {version}")

Environment variables have been loaded ;-)
Project Connection String: <...hub01-grp;mmai-swc-hub01-prj01>
azure-ai-projects library installed version: 1.0.0b10


# Create AI Foundry Project Client

In [2]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import FunctionTool, ToolOutput # <<<<<<<<<<<<<<< SPECIFIC FOR FUNCTION CALLING
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=project_connection_string
)

project_client.scope

{'subscription_id': 'eca2eddb-0f0c-4351-a634-52751499eeea',
 'resource_group_name': 'mmai-swc-hub01-grp',
 'project_name': 'mmai-swc-hub01-prj01'}

# Define some custom functions

In [3]:
from datetime import date

def save_file(joke:str) -> str:
    """
    Saves a text file using the textual content passed in the joke variable.
    Args:
        joke (str): The text content to be saved into the file. 
        
    Returns:
        Optional[str]: Returns 'success: file saved' on success, otherwise returns an error message.
    """
    file_name = "joke.txt"
    try:
        with open(file_name, "w") as f:
            f.write(joke)
        return "success: file saved"
    except Exception as e:
        return f"error: {str(e)}"


def get_flights(date_1:date, date_2:date) -> str:
    """ Returns the number of flights in a date interval  """
    import json
    from dateutil.parser import parse
    flights = {
        "flights": abs((parse(date_2) - parse(date_1)).days) 
    }
    return json.dumps(flights)


def my_cat_born_date() -> str:
    """ Returns my cat's born date """
    import datetime, random, json
    from dateutil.relativedelta import relativedelta
    
    # Calculate the date as ten years ago  
    ten_years_ago = datetime.date.today() - relativedelta(years=10) 
    
    cat_born_date = {
        "cat_born_date": ten_years_ago.strftime("%Y-%m-%d")
    }
    return json.dumps(cat_born_date)


def send_email(to:str, subject:str, body:str) -> str:
    """ Sends an email """
    import requests, json
    url = 'https://prod-18.swedencentral.logic.azure.com:443/workflows/4f7a19b041e04a9e8ea47303e1af503c/triggers/When_a_HTTP_request_is_received/paths/invoke?api-version=2016-10-01&sp=%2Ftriggers%2FWhen_a_HTTP_request_is_received%2Frun&sv=1.0&sig=TX-eDahoU_QIEOjw9qOXjRyPNqA9s4IVkd0osbsyzzI'  

    headers = {
        'Content-Type': 'application/json'
    }

    data = {
        "to": to,
        "subject": subject,
        "body": body
    }
    response = {"response": str(requests.post(url, headers=headers, data=json.dumps(data)))}
    return json.dumps(response)

# Consolidate the custom functions into a single set

In [4]:
from typing import Any, Callable, Set

user_functions: Set[Callable[..., Any]] = {
    save_file, 
    get_flights, 
    my_cat_born_date,
    send_email
}

user_functions

{<function __main__.get_flights(date_1: datetime.date, date_2: datetime.date) -> str>,
 <function __main__.my_cat_born_date() -> str>,
 <function __main__.save_file(joke: str) -> str>,
 <function __main__.send_email(to: str, subject: str, body: str) -> str>}

# Just for testing: use the `user_function` set to call its functions

In [5]:
for function in user_functions:    
    if function.__name__ == "my_cat_born_date":
        result = function()
    elif function.__name__ == "save_file": 
        result = function("This is the content")
    elif function.__name__ == "get_flights": 
        result = function("2015-01-01", "2021-04-04")
    elif function.__name__ == "send_email": 
        result = function("mauromi@microsoft.com", "email from Python Agent", "body")

    print(f"{function.__name__}() returned {result}")

my_cat_born_date() returned {"cat_born_date": "2015-05-27"}
send_email() returned {"response": "<Response [200]>"}
get_flights() returned {"flights": 2285}
save_file() returned success: file saved


# Initialize `FunctionTool`

In [6]:
functions = FunctionTool(user_functions)
print(f"functions.definitions: {functions.definitions}")
print(f"\nfunctions.resources: {functions.resources}")

functions.definitions: [{'type': 'function', 'function': {'name': 'my_cat_born_date', 'description': "Returns my cat's born date ", 'parameters': {'type': 'object', 'properties': {}, 'required': []}}}, {'type': 'function', 'function': {'name': 'send_email', 'description': 'Sends an email ', 'parameters': {'type': 'object', 'properties': {'to': {'type': 'string', 'description': 'No description'}, 'subject': {'type': 'string', 'description': 'No description'}, 'body': {'type': 'string', 'description': 'No description'}}, 'required': ['to', 'subject', 'body']}}}, {'type': 'function', 'function': {'name': 'get_flights', 'description': 'Returns the number of flights in a date interval  ', 'parameters': {'type': 'object', 'properties': {'date_1': {'type': 'string', 'description': 'No description'}, 'date_2': {'type': 'string', 'description': 'No description'}}, 'required': ['date_1', 'date_2']}}}, {'type': 'function', 'function': {'name': 'save_file', 'description': 'Saves a text file using 

In [7]:
# definitions pretty print
def format_tool_definitions(definitions):
    def custom_serializer(obj):
        return obj.__dict__ if hasattr(obj, '__dict__') else str(obj)

    function_dict = {item['function']['name']: item['function'] for item in functions.definitions}

    return json.dumps(function_dict, indent=2, default=custom_serializer)

print(format_tool_definitions (functions.definitions))

{
  "my_cat_born_date": {
    "_data": {
      "name": "my_cat_born_date",
      "description": "Returns my cat's born date ",
      "parameters": {
        "type": "object",
        "properties": {},
        "required": []
      }
    }
  },
  "send_email": {
    "_data": {
      "name": "send_email",
      "description": "Sends an email ",
      "parameters": {
        "type": "object",
        "properties": {
          "to": {
            "type": "string",
            "description": "No description"
          },
          "subject": {
            "type": "string",
            "description": "No description"
          },
          "body": {
            "type": "string",
            "description": "No description"
          }
        },
        "required": [
          "to",
          "subject",
          "body"
        ]
      }
    }
  },
  "get_flights": {
    "_data": {
      "name": "get_flights",
      "description": "Returns the number of flights in a date interval  ",
      "pa

# Create AI Foundry Agent

In [8]:
agent = project_client.agents.create_agent(
    model=model_name,
    name="functiontool-agent",
    instructions="You are a helpful assistant",
    tools=functions.definitions
)

agent.items

<bound method _MyMutableMapping.items of {'id': 'asst_Lka3PohT856pobZDFhUKYEts', 'object': 'assistant', 'created_at': 1748337939, 'name': 'functiontool-agent', 'description': None, 'model': 'gpt-4o', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'function', 'function': {'name': 'my_cat_born_date', 'description': "Returns my cat's born date ", 'parameters': {'type': 'object', 'properties': {}, 'required': []}, 'strict': False}}, {'type': 'function', 'function': {'name': 'send_email', 'description': 'Sends an email ', 'parameters': {'type': 'object', 'properties': {'to': {'type': 'string', 'description': 'No description'}, 'subject': {'type': 'string', 'description': 'No description'}, 'body': {'type': 'string', 'description': 'No description'}}, 'required': ['to', 'subject', 'body']}, 'strict': False}}, {'type': 'function', 'function': {'name': 'get_flights', 'description': 'Returns the number of flights in a date interval  ', 'parameters': {'type': 'object', 'proper

# Create the thread and attach a new message to it

In [9]:
# Create a thread
thread = project_client.agents.create_thread()
print(f"Created thread: {thread}\n")

# Add a user message to the thread
message = project_client.agents.create_message(
    thread_id=thread.id, 
    role="user", 
    content="Please write into a file the nr of flights between my cat born date and Easter 2021. Send the answer to mauromi@microsoft.com, also.",
)
print(f"Created message: {message}")

Created thread: {'id': 'thread_uM2aVH9blTE6GySa6jGrsyEc', 'object': 'thread', 'created_at': 1748337940, 'metadata': {}, 'tool_resources': {}}

Created message: {'id': 'msg_zDsFC0srIbBm6044ReJk5ait', 'object': 'thread.message', 'created_at': 1748337941, 'assistant_id': None, 'thread_id': 'thread_uM2aVH9blTE6GySa6jGrsyEc', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'Please write into a file the nr of flights between my cat born date and Easter 2021. Send the answer to mauromi@microsoft.com, also.', 'annotations': []}}], 'attachments': [], 'metadata': {}}


# Run the agent synchronously and check the status looping until the run stops
**Note**: if you get "incomplete" due to content filtering, please try running the same cell again

In [11]:
%%time

run = project_client.agents.create_run(thread_id=thread.id, agent_id=agent.id)

run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)

# run.status or run["status"]? it's up to you, but be aware to use it 
# use the run as json object, or as azure.ai.projects.models._models.ThreadRun
while run["status"] in ['queued', 'in_progress', 'cancelling']: # here I use the run as a json object
    time.sleep(1)
    run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)

if run["status"] == "failed": 
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")
elif run["status"] == "incomplete":
    print(f'"Incomplete" status returned due to the following reason: <{run["incomplete_details"]["reason"]}>. You may try to run this cell again.\n')
else:
    print(f"Run details:\n{run}\n\nRun finished with status: <{run.status}>. Please proceed by running the next cell.\n")

Run details:
{'id': 'run_XgAhc8yDZKyjSitYnyth48un', 'object': 'thread.run', 'created_at': 1748337957, 'assistant_id': 'asst_Lka3PohT856pobZDFhUKYEts', 'thread_id': 'thread_uM2aVH9blTE6GySa6jGrsyEc', 'status': 'requires_action', 'started_at': 1748337957, 'expires_at': 1748373957, 'cancelled_at': None, 'failed_at': None, 'completed_at': None, 'required_action': {'type': 'submit_tool_outputs', 'submit_tool_outputs': {'tool_calls': [{'id': 'call_gcxy5yIscUIzQaDX9DTUXix6', 'type': 'function', 'function': {'name': 'my_cat_born_date', 'arguments': '{}'}}]}}, 'last_error': None, 'model': 'gpt-4o', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'function', 'function': {'name': 'my_cat_born_date', 'description': "Returns my cat's born date ", 'parameters': {'type': 'object', 'properties': {}, 'required': []}, 'strict': False}}, {'type': 'function', 'function': {'name': 'send_email', 'description': 'Sends an email ', 'parameters': {'type': 'object', 'properties': {'to': {'type'

# KEEP RUNNING this cells until run.status == 'COMPLETED'

In [14]:
# just for checking: analyze the current status
import time

run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)
print(f"Initial run status: {run.status}")
print(f"\nRequired action(s): {run.required_action}")
print(f"\nWe need to run {len(run.required_action.submit_tool_outputs.tool_calls)} tool call(s): {run.required_action.submit_tool_outputs.tool_calls}")

i = 0
tool_outputs = []
for tool_call in run.required_action.submit_tool_outputs.tool_calls:
    i += 1
    output = functions.execute(tool_call)
    print(f"\n{i} - Executing tool_call {tool_call.function.name} ({tool_call.id}) >>> output: {output}")
    tool_outputs.append(
        ToolOutput(
            tool_call_id=tool_call.id,
            output=output
        )
    )
    
run = project_client.agents.submit_tool_outputs_to_run(
    thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs
)

while run["status"] in ["queued", "in_progress"]:
    time.sleep(1)
    run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)
    print(f"\nRun status: {run.status}")

if run["status"] == "requires_action":
    print("\nPlease run this cell again until all required actions are executed.")
elif run["status"] == "completed":
    print("\nGreat job! All required actions are completed. Please move on to the next cell.")

Initial run status: RunStatus.REQUIRES_ACTION

Required action(s): {'type': 'submit_tool_outputs', 'submit_tool_outputs': {'tool_calls': [{'id': 'call_zcyy0EW9t2QUFyyUJQYkMdHa', 'type': 'function', 'function': {'name': 'save_file', 'arguments': '{"joke": "The number of flights between my cat\'s birth date (2015-05-27) and Easter 2021 (2021-04-04) is: 2139."}'}}, {'id': 'call_sEcWwZljK05TSyRaVHAku0fw', 'type': 'function', 'function': {'name': 'send_email', 'arguments': '{"to": "mauromi@microsoft.com", "subject": "Number of Flights Information", "body": "The number of flights between my cat\'s birth date (2015-05-27) and Easter 2021 (2021-04-04) is: 2139."}'}}]}}

We need to run 2 tool call(s): [{'id': 'call_zcyy0EW9t2QUFyyUJQYkMdHa', 'type': 'function', 'function': {'name': 'save_file', 'arguments': '{"joke": "The number of flights between my cat\'s birth date (2015-05-27) and Easter 2021 (2021-04-04) is: 2139."}'}}, {'id': 'call_sEcWwZljK05TSyRaVHAku0fw', 'type': 'function', 'function'

# Fetch messages from the thread after the agent run execution

In [15]:
from azure.ai.projects.models import MessageTextContent, MessageImageFileContent

if run.status == 'completed':    
    messages = project_client.agents.list_messages(thread_id=thread.id)
    messages_nr = len(messages.data)
    print(f"Here are the {messages_nr} messages:\n")
    
    for i, message in enumerate(reversed(messages.data), 1):
        j = 0
        print(f"\n===== MESSAGE {i} =====")
        for c in message.content:
            j +=1
            if (type(c) is MessageImageFileContent):
                print(f"\nCONTENT {j} (MessageImageFileContent) --> image_file id: {c.image_file.file_id}")
            elif (type(c) is MessageTextContent):
                print(f"\nCONTENT {j} (MessageTextContent) --> Text: {c.text.value}")
                for a in c.text.annotations:
                    print(f">>> Annotation in MessageTextContent {j} of message {i}: {a.text}\n")

else:
    print(f"Sorry, I can't proceed because the run status is {run.status}")

Here are the 3 messages:


===== MESSAGE 1 =====

CONTENT 1 (MessageTextContent) --> Text: Please write into a file the nr of flights between my cat born date and Easter 2021. Send the answer to mauromi@microsoft.com, also.

===== MESSAGE 2 =====

CONTENT 1 (MessageTextContent) --> Text: I'm sorry, but I cannot assist with that request.

===== MESSAGE 3 =====

CONTENT 1 (MessageTextContent) --> Text: The number of flights between your cat's birth date (2015-05-27) and Easter 2021 (2021-04-04) is 2139. 

- This information has been written into a file.
- An email with the same details was sent to mauromi@microsoft.com.


# Run Steps

In [16]:
run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)

print(f'Nr of run step(s): {len(run_steps["data"])}\n')
i=0
for rs in run_steps["data"]:
    i += 1
    print(f"Run step {i}: {rs}", '\n')

Nr of run step(s): 4

Run step 1: {'id': 'step_iPI7kfhCHYVqTgp750mfstJo', 'object': 'thread.run.step', 'created_at': 1748337994, 'run_id': 'run_XgAhc8yDZKyjSitYnyth48un', 'assistant_id': 'asst_Lka3PohT856pobZDFhUKYEts', 'thread_id': 'thread_uM2aVH9blTE6GySa6jGrsyEc', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1748337995, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_g2Xzj9oWkBD0SOmv8TS3Ndpy'}}, 'usage': {'prompt_tokens': 692, 'completion_tokens': 66, 'total_tokens': 758, 'prompt_token_details': {'cached_tokens': 0}}} 

Run step 2: {'id': 'step_T6waB5UUDqQItcCB49jFPxVv', 'object': 'thread.run.step', 'created_at': 1748337974, 'run_id': 'run_XgAhc8yDZKyjSitYnyth48un', 'assistant_id': 'asst_Lka3PohT856pobZDFhUKYEts', 'thread_id': 'thread_uM2aVH9blTE6GySa6jGrsyEc', 'type': 'tool_calls', 'status': 'completed', 'cancelled_at': None, 'completed_at': 17

# Collect all resources for this project

In [17]:
all_agents = list_all_agents(project_client=project_client)
print(all_agents["summary"])

all_threads = list_all_threads(project_client)
print(all_threads["summary"])

all_files = list_all_files(project_client)
print(all_files["summary"])

all_runs = list_all_runs(project_client)
print(all_runs["summary"])

all_runsteps=list_all_runsteps(project_client)
print(all_runsteps["summary"])

all_messages = list_all_messages(project_client)
print(all_messages["summary"])

all_vectorstores = list_all_vectorstores(project_client=project_client)
print(all_vectorstores["summary"])

1 agents in project <mmai-swc-hub01-prj01>
1 threads in project <mmai-swc-hub01-prj01>
0 files in project <mmai-swc-hub01-prj01>
2 runs in 1 threads of project <mmai-swc-hub01-prj01>
5 run steps in 2 pairs of (thread, run) of project <mmai-swc-hub01-prj01>
3 messages in 1 threads of project <mmai-swc-hub01-prj01>
0 vector stores in project <mmai-swc-hub01-prj01>


# Teardown for all resources

In [18]:
# delete all vector stores

i=0
for vector_store in all_vectorstores["content"]["data"]:
    i += 1
    project_client.agents.delete_vector_store(vector_store_id=vector_store.id)
    print(f"{i} - Vector store <{vector_store.id}> has been deleted")

all_vectorstores = list_all_vectorstores(project_client=project_client)

print(f"Vector stores deleted: {i}\n")

Vector stores deleted: 0



In [19]:
# delete all files

i=0
for file in all_files['content']['data']:
    i += 1
    project_client.agents.delete_file(file_id=file.id)
    print(f"{i} - File <{file.filename}> ({file.id}) has been deleted")

all_files = list_all_files(project_client)

print(f"Files deleted: {i}\n")

Files deleted: 0



In [20]:
# delete all threads

i=0
for thread in all_threads["content"]["data"]:
    i += 1
    project_client.agents.delete_thread(thread_id=thread.id)
    print(f"{i} - Thread <{thread.id}> has been deleted")

all_threads = list_all_threads(project_client)

print(f"Threads deleted: {i}\n")

1 - Thread <thread_uM2aVH9blTE6GySa6jGrsyEc> has been deleted
Threads deleted: 1



In [21]:
# delete all agents

i=0
for agent in all_agents["content"]["data"]:
    i += 1
    project_client.agents.delete_agent(agent_id=agent.id)
    print(f"{i} - Agent <{agent.id}> has been deleted")

all_agents = list_all_agents(project_client=project_client)

print(f"Agents deleted: {i}\n")

1 - Agent <asst_Lka3PohT856pobZDFhUKYEts> has been deleted
Agents deleted: 1



# HIC SUNT LEONES